<a href="https://colab.research.google.com/github/seanigens/IntroDataViz/blob/main/PracticeCh13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
file_path = "/content/drive/My Drive/S544/AmesHousing.csv"
ames = pd.read_csv(file_path)

In [7]:
lr = LinearRegression()

X = ames[["Gr Liv Area", "TotRms AbvGrd"]]
y = ames["SalePrice"]



X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train_s = (X_train - X_train.mean())/X_train.std()

lr_fitted = lr.fit(X_train_s, y_train)
lr_fitted.coef_

array([ 71091.51557077, -17067.53559838])

In [10]:
X = ames[["Gr Liv Area", "TotRms AbvGrd", "Bldg Type"]]
y = ames["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline_1 = Pipeline([
    ('features', ColumnTransformer([
        ('interaction', PolynomialFeatures(interaction_only=True, include_bias=False), ['Gr Liv Area', 'TotRms AbvGrd'])
    ], remainder='drop')),
    ('linear_reg', LinearRegression())
])

pipeline_2 = Pipeline([
    ('features', ColumnTransformer([
        ('dummify', OneHotEncoder(), ['Bldg Type']),
        ('interaction', PolynomialFeatures(interaction_only=True, include_bias=False), ['Gr Liv Area', 'TotRms AbvGrd'])
    ], remainder='passthrough')),
    ('linear_reg', LinearRegression())
])

pipeline_3 = Pipeline([
    ('features', ColumnTransformer([
        ('dummify', OneHotEncoder(), ['Bldg Type']),
        ('interaction', PolynomialFeatures(interaction_only=True, include_bias=False), ['Gr Liv Area'])
    ], remainder='passthrough')),
    ('linear_reg', LinearRegression())
])

pipeline_4 = Pipeline([
    ('features', ColumnTransformer([
        ('dummify', OneHotEncoder(), ['Bldg Type']),
        ('poly_size', PolynomialFeatures(degree=5, include_bias=False), ['Gr Liv Area']),
        ('poly_rooms', PolynomialFeatures(degree=5, include_bias=False), ['TotRms AbvGrd'])
    ], remainder='passthrough')),
    ('linear_reg', LinearRegression())
])

pipelines = [pipeline_1, pipeline_2, pipeline_3, pipeline_4]
for pipe in pipelines:
    pipe.fit(X_train, y_train)

rmse_values = []
for pipe in pipelines:
    predictions = pipe.predict(X_test)
    rmse = mean_squared_error(y_test, predictions, squared=False)
    rmse_values.append(rmse)

best_model_index = rmse_values.index(min(rmse_values))
best_model_rmse = rmse_values[best_model_index]

best_model_index, best_model_rmse

(2, 59589.20317423393)

In [13]:
pipeline = Pipeline([
    ('features', ColumnTransformer([
        ('poly_size', PolynomialFeatures(include_bias=False), ['Gr Liv Area']),
        ('poly_rooms', PolynomialFeatures(include_bias=False), ['TotRms AbvGrd']),
        ('dummify', OneHotEncoder(), ['Bldg Type'])
    ], remainder='passthrough')),
    ('linear_reg', LinearRegression())
])

paramiter = {
    'features__poly_size__degree': range(1, 11),
    'features__poly_rooms__degree': range(1, 11)
}

search = GridSearchCV(pipeline, paramiter, cv=5, scoring='neg_mean_squared_error', verbose=1)

search.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('features',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('poly_size',
                                                                         PolynomialFeatures(include_bias=False),
                                                                         ['Gr '
                                                                          'Liv '
                                                                          'Area']),
                                                                        ('poly_rooms',
                                                                         PolynomialFeatures(include_bias=False),
                                                                         ['TotRms '
                                                                          'AbvGrd']),
                                                                        ('dummify',
                                                                         OneHotEncoder(),
                                                                         ['Bldg '
                                                                          'Type'])])),
                                       ('linear_reg', LinearRegression())]),
             param_grid={'features__poly_rooms__degree': range(1, 11),
                         'features__poly_size__degree': range(1, 11)},
             scoring='neg_mean_squared_error', verbose=1)